In [1]:
import pandas as pd
import numpy as np

# Read in the data
df = pd.read_csv('archive.zip', compression='zip')

df.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1


In [2]:
#Se toman solo las palabras en inglés
df_new = df[df["learning_language"]=="en"]

#Se crean nuevas columnas 
df_new.loc[:,("times_reviewed",)] = df_new[["history_seen","session_seen"]].sum(axis=1)
df_new.loc[:,("times_correct",)] = df_new[["history_correct","session_correct"]].sum(axis=1)
df_new.loc[:,("recall",)] = df_new["session_correct"].div(df_new["session_seen"])

#Se transforman los datos al formato necesario:
#Transformar a 1 y 0 
df_new.loc[:,("recall",)][df_new["recall"]>=0.5]=1
df_new.loc[:,("recall",)][df_new["recall"]<0.5]=0
#Transformar a dia
df_new.loc[:,("delta",)]=df_new["delta"]/86400
df_new.loc[:,("delta",)] = np.floor(df_new["delta"])

df_new = df_new.drop(['p_recall','timestamp','ui_language',"lexeme_id","learning_language","history_seen",
                      "history_correct","session_correct","session_seen"],axis='columns')
print(len(df_new))
df_new.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18232\1562030414.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00413194 0.00413194 0.00413194 ... 0.00425926 0.00425926 0.00425926]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_new.loc[:,("delta",)]=df_new["delta"]/86400


5014791


,delta,user_id,lexeme_string,times_reviewed,times_correct,recall
87,0.0,u:dwbJ,definition/definition<n><sg>,19,19,1.0
88,0.0,u:dwbJ,surface/surface<n><sg>,22,22,1.0
89,0.0,u:dwbJ,scale/scale<n><sg>,24,23,1.0
90,0.0,u:dwbJ,temperature/temperature<n><sg>,49,40,0.8
91,0.0,u:dwbJ,distance/distance<n><sg>,24,23,1.0


In [3]:
#Se eliminan los usuarios con menos de 10 palabras aprendidas
#freq = df_new["user_id"].value_counts()
#ind = freq[freq <= 10].index
#df_new = df_new[~df_new["user_id"].isin(ind)]
#print(len(df_new))
#df_new["user_id"].value_counts()

In [4]:
#Crear columna de palabras a partir del campo de caracteres "lexeme_string"
df_new["word"]=""
df_new["word"] = df_new['lexeme_string'].str.extract(r"(^[a-zA-Z]+)")

indexes = df_new[df_new["word"].isna()].index
df_new.loc[indexes,"word"] = df_new.loc[indexes, :]["lexeme_string"].str.extract(r"/([a-zA-Z]+)")[0]

indexes = df_new[df_new["word"].isna()].index
df_new = df_new.drop(indexes,axis="index")
df_new = df_new.drop(["lexeme_string","user_id"],axis='columns')
len(df_new)

4997499

In [5]:
#Eliminar palabras irrelevantes que puedan perjudicar el entrenamiento
del_words= df_new["word"].isin(["a","an","is","we","are","am","you","it","they","its","my","not","i",
                                "her","the","me","and","she","our","he","yes","do","his","to","does","their",
                                "no","him","us","at","them","or","nor"])
indexes = df_new[del_words].index
df_new = df_new.drop(indexes,axis="index")
print(len(df_new))
df_new["word"].value_counts()

3845333


word
eats            91640
eat             69605
drink           62365
man             60331
girl            59105
                ...  
actors              1
systems             1
organization        1
walls               1
fingers             1
Name: count, Length: 1779, dtype: int64

In [6]:
df_new.head()

,delta,times_reviewed,times_correct,recall,word
87,0.0,19,19,1.0,definition
88,0.0,22,22,1.0,surface
89,0.0,24,23,1.0,scale
90,0.0,49,40,0.8,temperature
91,0.0,24,23,1.0,distance


In [8]:
#Exportar
df_new.to_csv('duolingo.csv',index=False)